In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image

In [ ]:
def load(filename):
    file = open(filename, "r") 
    image_names = file.readlines()
    images = []
    labels = []
    counter = 1
    for name in image_names:
        label = name[:3]
        if label <= "001":
            im = Image.open("images/" + name.rstrip('\n'))
            H, W = im.size
            pixels = list(im.getdata())
            image = np.array([comp for pixel in pixels for comp in pixel]).reshape(H, W, 3)
            print(image.shape)
            images.append(image)
            labels.append(label)
            if counter >= 2:
                break
            counter += 1
            
#     print(len(images))
#     print(len(labels))
    return images, labels

images, labels = load('train.txt')

# print('Input shape: ' + str(image_data.shape))
# print('Labels shape: ' + str(label_data.shape))

# num_classes = 200

In [11]:
inputs = tf.placeholder(tf.float32, (None,None,3))
net = tf.identity(inputs, name='net')
net = tf.contrib.layers.conv2d(net, 20, 3, stride=2, activation_fn=tf.nn.relu)
# net = tf.contrib.layers.conv2d(net, 30, 3, stride=2, activation_fn=tf.nn.relu)
print(net)

tf.reset_default_graph()

def load(filename):
    file = open(filename, "r") 
    image_names = file.readlines()
    images = []
    labels = []
    counter = 1
    for name in image_names:
        label = name[:3]
        if label <= "001":
            im = Image.open("images/" + name.rstrip('\n'))
            H, W = im.size
            pixels = list(im.getdata())
            image = np.array([comp for pixel in pixels for comp in pixel]).reshape(H, W, 3)
            print(image.shape)
            images.append(image)
            labels.append(label)
            
            t_image = tf.convert_to_tensor(image)
            t_image = tf.to_float(image)
#             net = tf.identity(t_image, name='net')
#             net = tf.contrib.layers.conv2d(net, 20, 3, stride=2, activation_fn=tf.nn.relu)
#             print(net.shape)
            
            
            if counter >= 2:
                break
            counter += 1
            
            
#     print(len(images))
#     print(len(labels))
    return images, labels

images, labels = load('train.txt')

# print('Input shape: ' + str(image_data.shape))
# print('Labels shape: ' + str(label_data.shape))

# num_classes = 200

print( "Total number of variables used ", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]), '/', 500000 )



Tensor("Conv/Relu:0", shape=(?, ?, 20), dtype=float32)
(500, 336, 3)
Tensor("ToFloat:0", shape=(500, 336, 3), dtype=float32)
(415, 500, 3)
Tensor("ToFloat_1:0", shape=(415, 500, 3), dtype=float32)
Total number of variables used  0.0 / 500000


In [25]:
tf.reset_default_graph()

# todo: use tf.contrib.layers.conv2d for 3D filter
def conv_relu(input, kernel_shape, bias_shape):
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[2, 2, 2, 2], padding='SAME')
    return tf.nn.relu(conv + biases)

def my_image_filter(input_images):
    # Variables created here will be named "convX/weights", "convX/biases".
    with tf.variable_scope("conv1"):
        output = conv_relu(input_images, [5, 5, 3, 3], [3])
    with tf.variable_scope("conv2"):
        output = conv_relu(output, [5, 5, 3, 3], [3])
    with tf.variable_scope("conv3"):
        return conv_relu(output, [5, 5, 3, 3], [3])

def load(filename):
    file = open(filename, "r") 
    image_names = file.readlines()
    images = []
    labels = []
    counter = 1
    with tf.variable_scope("image_filters") as scope:
        for name in image_names:
            label = name[:3]
            if label <= "001":
                im = Image.open("images/" + name.rstrip('\n'))
                H, W = im.size
                pixels = list(im.getdata())
                image = np.array([comp for pixel in pixels for comp in pixel]).reshape(-1, H, W, 3)

                t_image = tf.convert_to_tensor(image)
                t_image = tf.to_float(image)

                images.append(t_image)
                labels.append(label)
            
                print(image.shape)
                print(my_image_filter(t_image))
                scope.reuse_variables()

                if counter >= 2:
                    break
                counter += 1
#     print(len(images))
#     print(len(labels))
    return images, labels

images, labels = load('train.txt')

# print('Input shape: ' + str(image_data.shape))
# print('Labels shape: ' + str(label_data.shape))

# num_classes = 200

print( "Total number of variables used ", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]), '/', 500000 )






(1, 500, 336, 3)
Tensor("image_filters/conv2/Relu:0", shape=(1, 125, 84, 3), dtype=float32)
(1, 415, 500, 3)
Tensor("image_filters/conv2_1/Relu:0", shape=(1, 104, 125, 3), dtype=float32)
2
2
Total number of variables used  456 / 500000
